In [50]:
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
import http.client
import requests
import socket
from subsense_utils import time_util

base_url = 'http://localhost:8079'
base_url = 'http://localhost:8081'
url_path = '/backfill/reservoir_level'

In [51]:
start = 1562630400000
end = 1543708800000
end = time_util.get_now()

def create_time_spans(start, end, splits=None, split_size=None):
    if split_size and splits:
        raise ValueError('Only one of splits and split_size should be set. ' + 
                         ' Got splits: %s and split_size: %s', 
                         splits, 
                         split_size)
    # if splits:
    #    split_size = 100
        
    if not split_size:
        split_size = time_util.ONE_DAY * 10
        
    ranges = [(n, min(n+split_size, end)) for n in range(start, end, split_size)]

    return ranges

In [52]:
ranges = create_time_spans(start, end)
urls = []
for span_start, span_end in ranges:
    urls.append(f'{base_url}{url_path}/hpu?start={span_start}&end={span_end}')
    urls.append(f'{base_url}{url_path}/prod?start={span_start}&end={span_end}')
    urls.append(f'{base_url}{url_path}/ssiv?start={span_start}&end={span_end}')

In [54]:
def get_it(url):
    try:
        
        # connection = http.client.HTTPSConnection(url, timeout=2)
        r = requests.get(url)
        #connection.request("GET", "/")
        #response = connection.getresponse()
        
        print('='*10)
        print('url:', url)
        print('r.status_code: ',r.status_code)
        return r.status_code
        # return response.read()
    except socket.timeout:
        # in a real world scenario you would probably do stuff if the
        # socket goes into timeout
        pass

print('running for:',len(urls))
with PoolExecutor(max_workers=20) as executor:
    for _ in executor.map(get_it, urls):
        pass

http://localhost:8081/backfill/reservoir_level/hpu?start=1562630400000&end=1563494400000
200
http://localhost:8081/backfill/reservoir_level/hpu?start=1563494400000&end=1564358400000
200
http://localhost:8081/backfill/reservoir_level/prod?start=1562630400000&end=1563494400000
200
http://localhost:8081/backfill/reservoir_level/prod?start=1564358400000&end=1565222400000
200
http://localhost:8081/backfill/reservoir_level/ssiv?start=1562630400000&end=1563494400000
200
http://localhost:8081/backfill/reservoir_level/hpu?start=1564358400000&end=1565222400000
200
http://localhost:8081/backfill/reservoir_level/hpu?start=1565222400000&end=1566048599416
200
http://localhost:8081/backfill/reservoir_level/ssiv?start=1565222400000&end=1566048599416
200
http://localhost:8081/backfill/reservoir_level/ssiv?start=1564358400000&end=1565222400000
200
http://localhost:8081/backfill/reservoir_level/prod?start=1565222400000&end=1566048599416
200


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))